In [1]:
import pandas as pd

In [2]:
df_merra2 = pd.read_csv("data/MERRA2_data.csv", skiprows=9)
df_meteorologicos_porto_alegre = pd.read_csv("data/dados_meteorologicos_porto_alegre.csv")
df_meteorologicos_serra_gaucha = pd.read_csv("data/dados_meteorologicos_serra_gaucha.csv")

df_cai_nivel = pd.read_csv("data/df_cai_nivel.csv")
df_gravatai_nivel = pd.read_csv("data/df_gravatai_nivel.csv")
df_guaiba_nivel = pd.read_csv("data/df_guaiba_nivel.csv")
df_jacui_nivel = pd.read_csv("data/df_jacui_nivel.csv")
df_sinos_nivel = pd.read_csv("data/df_sinos_nivel.csv")
df_taquari_nivel = pd.read_csv("data/df_taquari_nivel.csv")

In [3]:
# df_meteorologicos_serra_gaucha = df_meteorologicos_serra_gaucha.drop(columns=["snow", "wpgt", "tsun"])
# df_meteorologicos_porto_alegre = df_meteorologicos_porto_alegre.drop(columns=["snow", "wpgt", "tsun"])
# df_meteorologicos_serra_gaucha.columns = ["date", "tavg_serra", "tmin_serra", "tmax_serra", "prcp_serra", "wdir_serra", "wspd_serra", "pres_serra"]
# df_meteorologicos_porto_alegre.columns = ["date", "tavg_poa", "tmin_poa", "tmax_poa", "prcp_poa", "wdir_poa", "wspd_poa", "pres_poa"]

In [4]:
df_cai_nivel.columns = ["date", "water_level_cm_cai", "hourly_rain_mm_cai"]
df_gravatai_nivel.columns = ["date", "water_level_cm_gravatai", "hourly_rain_mm_gravatai"]
df_guaiba_nivel.columns = ["date", "water_level_cm_guaiba", "hourly_rain_mm_guaiba"]
df_jacui_nivel.columns = ["date", "water_level_cm_jacui", "hourly_rain_mm_jacui"]
df_sinos_nivel.columns = ["date", "water_level_cm_sinos", "hourly_rain_mm_sinos"]
df_taquari_nivel.columns = ["date", "water_level_cm_taquari", "hourly_rain_mm_taquari"]

In [5]:
# df_merra2["Date"] = pd.to_datetime(df_merra2["Date"])

df_cai_nivel["date"] = pd.to_datetime(df_cai_nivel["date"])
df_gravatai_nivel["date"] = pd.to_datetime(df_gravatai_nivel["date"])
df_guaiba_nivel["date"] = pd.to_datetime(df_guaiba_nivel["date"])
df_jacui_nivel["date"] = pd.to_datetime(df_jacui_nivel["date"])
df_sinos_nivel["date"] = pd.to_datetime(df_sinos_nivel["date"])
df_taquari_nivel["date"] = pd.to_datetime(df_taquari_nivel["date"])

# 
# df_cai_nivel = df_cai_nivel.sort_values(by="date").reset_index(drop=True)
df_gravatai_nivel = df_gravatai_nivel.sort_values(by="date").reset_index(drop=True)
df_guaiba_nivel = df_guaiba_nivel.sort_values(by="date").reset_index(drop=True)
df_jacui_nivel = df_jacui_nivel.sort_values(by="date").reset_index(drop=True)
df_sinos_nivel = df_sinos_nivel.sort_values(by="date").reset_index(drop=True)
df_taquari_nivel = df_taquari_nivel.sort_values(by="date").reset_index(drop=True)

In [6]:
from pandas.core.frame import DataFrame
def to_hourly(df: DataFrame, river_name: str) -> DataFrame:
    df.set_index('date', inplace=True)

    # Agrupando para frequência horária
    df = df.resample('H').agg({
        f'water_level_cm_{river_name}': 'mean',
        f'hourly_rain_mm_{river_name}': 'sum'
    }).reset_index()
    return df


In [7]:
df_gravatai_hourly = to_hourly(df=df_gravatai_nivel, river_name="gravatai")
df_guaiba_hourly = to_hourly(df=df_guaiba_nivel, river_name="guaiba")
df_jacui_hourly = to_hourly(df=df_jacui_nivel, river_name="jacui")
df_sinos_hourly = to_hourly(df=df_sinos_nivel, river_name="sinos")
df_taquari_hourly = to_hourly(df=df_taquari_nivel, river_name="taquari")

In [8]:
df_sinos_hourly

,date,water_level_cm_sinos,hourly_rain_mm_sinos
0,2024-04-15 00:00:00,122.00,0.0
1,2024-04-15 01:00:00,120.50,0.0
2,2024-04-15 02:00:00,119.50,0.0
3,2024-04-15 03:00:00,118.50,0.0
4,2024-04-15 04:00:00,118.00,0.0
...,...,...,...
1097,2024-05-30 17:00:00,481.25,0.0
1098,2024-05-30 18:00:00,480.75,0.0
1099,2024-05-30 19:00:00,480.50,0.0
1100,2024-05-30 20:00:00,480.00,0.0


## Merra2

In [9]:
df_merra2['Time (UTC)'] = pd.to_datetime(df_merra2['Time (UTC)'], format='%H:%M:%S').dt.floor('H').dt.time

# Concatenar as colunas Date e Time (UTC)
df_merra2['datetime_utc'] = pd.to_datetime(df_merra2['Date'] + ' ' + df_merra2['Time (UTC)'].astype(str), format='%d/%m/%Y %H:%M:%S')

# Converter de UTC para o fuso horário de Brasília (BRT)
df_merra2['datetime_brasilia'] = df_merra2['datetime_utc'].dt.tz_localize('UTC').dt.tz_convert('America/Sao_Paulo')

# Remover o fuso horário para manter o formato YYYY-MM-DD HH:MM:SS
df_merra2['datetime_brasilia'] = df_merra2['datetime_brasilia'].dt.tz_localize(None)


In [10]:
df_merra2 = df_merra2[(df_merra2["datetime_brasilia"]>="2024-04-15 00:00:00")&(df_merra2["datetime_brasilia"]<="2024-05-30 21:00:00")]

In [11]:
df_merra2 = df_merra2.rename(columns={"datetime_brasilia": "date"})

In [12]:
df_merra2 = df_merra2[["date", "Wind Speed", "Wind Direction", "Temperature", "Pressure"]]

In [13]:
df_merra2.columns = ["date", "wind_speed_poa", "wind_direction_poa", "temperature_poa", "pressure_poa"]

In [14]:
df_merra2

,date,wind_speed_poa,wind_direction_poa,temperature_poa,pressure_poa
1776,2024-04-15 00:00:00,5.53,86.08,22.68,1007.94
1777,2024-04-15 01:00:00,5.74,91.70,22.56,1007.83
1778,2024-04-15 02:00:00,5.89,94.10,22.49,1007.66
1779,2024-04-15 03:00:00,6.29,90.87,22.54,1007.30
1780,2024-04-15 04:00:00,6.60,86.56,22.56,1007.03
...,...,...,...,...,...
2873,2024-05-30 17:00:00,1.89,164.64,16.33,1019.05
2874,2024-05-30 18:00:00,2.82,156.10,15.40,1019.62
2875,2024-05-30 19:00:00,3.30,140.79,14.55,1020.02
2876,2024-05-30 20:00:00,3.37,127.12,13.86,1020.37


In [15]:
dfs = [df_merra2, df_gravatai_hourly, df_jacui_hourly, df_sinos_hourly, df_taquari_hourly, df_guaiba_hourly]

# Utilizar merge sequencial para juntar todos os dataframes
df_merged = dfs[0]
for df in dfs[1:]:
    df_merged = df_merged.merge(df, on='date', how='outer')

In [16]:
df_merged = df_merged.interpolate(method='ffill', limit_direction='forward').copy()

In [21]:
df_merged.to_csv("data/df.csv", index=False)